In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import json
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

import time
import pickle

import handoff_eval

# Load data

In [ ]:
data_path = "../data/ai_ml_take_home"
ground_truth_path = os.path.join(data_path, "ground_truth")
model_outputs_path = os.path.join(data_path, "model_outputs")

In [ ]:
ground_truth_data = handoff_eval.load_json_files(ground_truth_path)
model_output_data = handoff_eval.load_json_files(model_outputs_path)

# Reverse engineering

to find how they used LLM to extract line-items estimations from a simple description (is it possible)?

- they use the input (description of the project with details and requirements)
- and a prompt to generate the estimated list of line-items

In [ ]:
import openai
import json
import re

def get_line_items(description):
    prompt = f"""
    Given the following bathroom remodel project description, \
    generate a structured list of line items with quantities, unit rates, and total costs.
    
    Description:
    {description}
    
    Expected output:
    - A JSON object with a "rows" array containing structured cost estimates.
    - Each row should have: sectionName, qty, rateUsd, rowTotalCostUsd, label, uom, and category.
    - Use common industry rates for cost estimation.
    """

    response = openai.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a construction cost estimator",
            },
            {"role": "user", "content": prompt},
        ],
        model="gpt-4-turbo",
        temperature=0,
    )
    
    message = response.choices[0].message.content
    
    # Extract the JSON part using regex
    json_pattern = r"\{.*\}"
    match = re.search(json_pattern, message, re.DOTALL)

    if match:
        json_string = match.group(0)
        try:
            # Ensure proper JSON formatting before parsing
            parsed_response = json.loads(json_string.strip())
            return parsed_response
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {e}")
            print("Raw JSON Extracted:", json_string)
            return "JSON is malformed"
    else:
        return "No JSON match found"

description = """
Bathroom Remodel project: 7'x10'-6", 8 ft ceilings
Demo a tub/shower combo
Replace with a complete shower system, with tiled walls, niche, and curb.
Replace tile floor with new tile.
New double vanity.
New Toilet.
Plumbing fixtures stay in place, no relocation or new rough-in needed.
Repaint whole room 3 coats.
Replace 4 light fixtures.
Shower Glass Enclosure excluded.
"""

# line_items = get_line_items(description)
# print(json.dumps(line_items, indent=4))

In [ ]:
# --- EXAMPLE OF OUTPUT:

# ```json
# {
#   "rows": [
#     {
#       "sectionName": "Demolition",
#       "qty": 1,
#       "rateUsd": 500,
#       "rowTotalCostUsd": 500,
#       "label": "Demolition of existing tub/shower combo",
#       "uom": "job",
#       "category": "Demolition"
#     },
#     {
#       "sectionName": "Shower Installation",
#       "qty": 1,
#       "rateUsd": 2500,
#       "rowTotalCostUsd": 2500,
#       "label": "Installation of complete shower system",
#       "uom": "job",
#       "category": "Plumbing"
#     },
#     {
#       "sectionName": "Tiling - Shower",
#       "qty": 80,
#       "rateUsd": 15,
#       "rowTotalCostUsd": 1200,
#       "label": "Tiling for shower walls",
#       "uom": "sq ft",
#       "category": "Finishes"
#     },
#     {
#       "sectionName": "Tiling - Floor",
#       "qty": 75,
#       "rateUsd": 12,
#       "rowTotalCostUsd": 900,
#       "label": "Tiling for bathroom floor",
#       "uom": "sq ft",
#       "category": "Finishes"
#     },
#     {
#       "sectionName": "Vanity Installation",
#       "qty": 1,
#       "rateUsd": 800,
#       "rowTotalCostUsd": 800,
#       "label": "Installation of new double vanity",
#       "uom": "job",
#       "category": "Cabinetry"
#     },
#     {
#       "sectionName": "Toilet Replacement",
#       "qty": 1,
#       "rateUsd": 400,
#       "rowTotalCostUsd": 400,
#       "label": "Replacement of toilet",
#       "uom": "job",
#       "category": "Plumbing"
#     },
#     {
#       "sectionName": "Painting",
#       "qty": 225,
#       "rateUsd": 3,
#       "rowTotalCostUsd": 675,
#       "label": "Repainting bathroom walls and ceiling",
#       "uom": "sq ft",
#       "category": "Painting"
#     },
#     {
#       "sectionName": "Light Fixtures",
#       "qty": 4,
#       "rateUsd": 150,
#       "rowTotalCostUsd": 600,
#       "label": "Replacement of light fixtures",
#       "uom": "each",
#       "category": "Electrical"
#     }
#   ]
# }
# ```
